In [1]:
import pandas as d
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from ModelMethods import *
from tqdm import tqdm

In [2]:
with open('data/institution_geo24.pkl', 'rb') as f:
    institution_geo = pickle.load(f)
institution_geo

,institution_id,city,region,country_code,country,latitude,longitude
0,I220871436,NaN,NaN,FR,NaN,48.848610,2.342780
1,I4210140407,Portland,Oregon,US,United States,45.525047,-122.672940
2,I4210121796,London,NaN,GB,United Kingdom,51.544495,-0.056045
3,I2800206933,Vienna,NaN,AT,Austria,48.204475,16.357422
4,I2802204158,Salem,Oregon,US,United States,44.886715,-123.031906
...,...,...,...,...,...,...,...
108136,I4394709273,Washington,NaN,NaN,United States,38.895110,-77.036370
108137,I4394709274,Delft,NaN,NaN,The Netherlands,52.006670,4.355560
108138,I4394709279,Washington,NaN,NaN,United States,38.895110,-77.036370
108139,I82930923,London,NaN,GB,United Kingdom,51.531994,-0.119311


In [3]:
institution_geo.dropna(subset=["city", 'country_code'], inplace=True)

ccframe = pd.read_csv('data/country-and-continent-codes-list-csv.csv')

def get_continent(country_code):
    
    if country_code == 'XK':
        con = 'EU'
    else:
        con =  str(ccframe.loc[ccframe['Two_Letter_Country_Code'] == country_code ]['Continent_Code'].values[0])
    if(con != 'nan'):
        return con
    else:
        return'NA'
        ## Pandas 好像会自动把美洲的code ”NA“ 转化成NaN，所以要特判一下
        
institution_geo['continent_code'] = institution_geo['country_code'].apply(get_continent)

In [4]:
# Change all TW to CN in the country_code column 
institution_geo.loc[institution_geo['country_code'] == 'TW', 'country_code'] = 'CN'
# Change all Taiwan to China in the country column
institution_geo.loc[institution_geo['country'] == 'Taiwan', 'country'] = 'China'

In [5]:
with open('data/movement_df_with_ld_validation_250121.pkl', 'rb') as f:
    movement_df = pickle.load(f)
movement_df

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,pubs_count
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NaN,CA,2,0,0,2,1,2,7
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NaN,US,1,0,0,1,1,1,24
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,1,24
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NaN,PK,3,1,5,1,1,7,24
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,3,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194551,A5097964981,2004,I4210139845,2004-03-01,I904495901,NaN,Bochum,DE,EU,DE,0,0,0,2,3,2,5
5194552,A5097991687,1993,I2801112126,1993-08-01,I4210122458,NaN,Bern,CH,EU,CH,0,0,0,2,2,2,4
5194553,A5098096471,1985,I4210159963,1985-11-01,I2801942218,NaN,Copenhagen,DK,EU,DK,1,1,0,1,2,2,3
5194554,A5098805673,2015,I80043,2015-12-01,I4210147769,NaN,Paris,FR,EU,ES,2,23,0,3,3,26,9


In [6]:
#Use related work, coauthors, and field to verify the movement
movement_df['verifiation_score'] = movement_df['work'] + movement_df['coauthors'] + movement_df['field']
verified_df = movement_df[movement_df['verifiation_score'] > 0 ]
len(verified_df) / len(movement_df)

0.9927653104519424

In [7]:
move_data = verified_df
move_data['move_time'] = pd.to_datetime(move_data['move_time'])

# def get_year(time):
#     return time.year

# move_data['move_year'] = move_data['move_time'].apply(get_year)
# move_data

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_1658/2035865764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move_data['move_time'] = pd.to_datetime(move_data['move_time'])


In [8]:
destinations = move_data['institution_id'].unique()
origins = move_data['prev_ins'].unique()

institution_geo = institution_geo[institution_geo['institution_id'].isin(destinations)|
                                  institution_geo['institution_id'].isin(origins)]
institution_geo

,institution_id,city,region,country_code,country,latitude,longitude,continent_code
484,I4210142806,Boone,North Carolina,US,United States,36.199036,-81.652270,NA
485,I4210103515,Rubano,NaN,IT,Italy,45.425476,11.789450,EU
487,I4210130444,Manchester,Missouri,US,United States,38.596786,-90.509660,NA
489,I4210117313,San Diego,California,US,United States,32.714620,-117.151380,NA
490,I4210124511,Philadelphia,Pennsylvania,US,United States,39.952330,-75.163790,NA
...,...,...,...,...,...,...,...,...
107143,I4210138289,Milton,Queensland,AU,Australia,-27.467762,153.006040,OC
107191,I4210143133,New Delhi,NaN,IN,India,28.624348,77.191010,AS
107348,I4210159722,Arlington,NaN,US,United States,38.881010,-77.104280,NA
107380,I4210163951,Jilin City,NaN,CN,China,43.850830,126.560280,AS


In [9]:
# replace NaN in continent_code in movedata with 'NA'
move_data['continent_code'] = move_data['continent_code'].fillna('NA')
move_data

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_1658/2089574962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  move_data['continent_code'] = move_data['continent_code'].fillna('NA')


,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,pubs_count,verifiation_score
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NA,CA,2,0,0,2,1,2,7,2
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NA,US,1,0,0,1,1,1,24,1
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,1,24,1
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NA,PK,3,1,5,1,1,7,24,7
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,3,88,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5194551,A5097964981,2004,I4210139845,2004-03-01,I904495901,NaN,Bochum,DE,EU,DE,0,0,0,2,3,2,5,2
5194552,A5097991687,1993,I2801112126,1993-08-01,I4210122458,NaN,Bern,CH,EU,CH,0,0,0,2,2,2,4,2
5194553,A5098096471,1985,I4210159963,1985-11-01,I2801942218,NaN,Copenhagen,DK,EU,DK,1,1,0,1,2,2,3,2
5194554,A5098805673,2015,I80043,2015-12-01,I4210147769,NaN,Paris,FR,EU,ES,2,23,0,3,3,26,9,26


# Attractiveness

In [11]:
continents = institution_geo['continent_code'].unique()
con_atr = pd.DataFrame(columns=continents)

for year in range(1960, 2021, 5):
    slice_data = move_data[ (move_data['move_year'] >= year) 
                           & (move_data['move_year']<(year+5))]
    con_atr_slice = dict()
    sum = len(slice_data)
    for continent in continents:
        con_atr_slice[continent] = len(slice_data[slice_data['continent_code'] == continent]) / sum
    df_slice = pd.DataFrame([con_atr_slice], index=['%d-%d'%(year, year+5)])
    con_atr = pd.concat([con_atr,df_slice])
con_atr
con_atr.to_csv('data/attrativeness_continent.csv')

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_77957/131141179.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  con_atr = pd.concat([con_atr,df_slice])


In [12]:
writer = pd.ExcelWriter('data/attractiveness_countries.xlsx')
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for continent in continents:
        cou_atr = attr_within_continent(move_data=move_data, 
                                        continent=continent)
        cou_atr.to_excel(writer, sheet_name=continent)
    writer.close()

In [13]:
writer = pd.ExcelWriter('data/attractiveness_cities.xlsx')
all_countries = move_data['country_code'].unique()

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for country in all_countries:
        city_atr = attr_within_country(move_data, country)
        city_atr.to_excel(writer, sheet_name=country)
    writer.close()

# Level Distances

In [14]:
move_data.head()

,author_id,move_year,institution_id,move_time,prev_ins,stay_time,city,country_code,continent_code,prev_country,level_distance,work,coauthors,field,domain,author_validated,pubs_count,verifiation_score
0,A5000000072,2019,I198243066,2019-01-01,I153718931,NaN,Heredia,CR,NA,CA,2,0,0,2,1,2,7,2
1,A5000000092,2002,I74760111,2002-11-23,I131249849,4057 days 00:00:00,Dallas,US,NA,US,1,0,0,1,1,1,24,1
2,A5000000092,2014,I16076960,2014-01-01,I74760111,2475 days 00:00:00,Islamabad,PK,AS,US,3,0,0,1,1,1,24,1
3,A5000000092,2020,I131249849,2020-10-11,I16076960,NaN,Corvallis,US,NA,PK,3,1,5,1,1,7,24,7
4,A5000000103,1992,I4210135003,1992-05-01,I88155538,11644 days 00:00:00,Pamplona,ES,EU,ES,0,0,1,2,2,3,88,3


In [15]:
ld_col = ['P(d=0)', 'P(d=1)', 'P(d=2)', 'P(d=3)']

writer = pd.ExcelWriter('data/level_distances.xlsx')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for country in all_countries:
        ld_df = pd.DataFrame(columns=ld_col)
        country_movement = move_data[move_data['prev_country']==country]
        for year in range(1960, 2021, 5):
            slice_data = country_movement[ (country_movement['move_year'] >= year) 
                                & (country_movement['move_year']<(year+5))]
            sum = len(slice_data)
            pld = [len(slice_data[slice_data['level_distance'] == i])/sum if sum !=0 else 0
                for i in range(4)]
            pld = pd.DataFrame([pld], columns=ld_col, 
                            index=['%d-%d'%(year, year+5)])
            ld_df = pd.concat([ld_df, pld], axis=0)
        ld_df.to_excel(writer, sheet_name = country)
    writer.close()        

# Movement data selected by publication counts.

In [10]:
print(len(move_data[move_data['pubs_count'] < 400]) / len(move_data))
move_data = move_data[move_data['pubs_count'] < 400]

0.9900792615826138


In [11]:
continents = institution_geo['continent_code'].unique()
con_atr = pd.DataFrame(columns=continents)

for year in range(1960, 2021, 5):
    slice_data = move_data[ (move_data['move_year'] >= year) 
                           & (move_data['move_year']<(year+5))]
    con_atr_slice = dict()
    sum = len(slice_data)
    for continent in continents:
        con_atr_slice[continent] = len(slice_data[slice_data['continent_code'] == continent]) / sum
    df_slice = pd.DataFrame([con_atr_slice], index=['%d-%d'%(year, year+5)])
    con_atr = pd.concat([con_atr,df_slice])

con_atr.to_csv('data/Double_verified_results/attrativeness_continent.csv')

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_1658/3422456219.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  con_atr = pd.concat([con_atr,df_slice])


In [12]:
writer = pd.ExcelWriter('data/Double_verified_results/attractiveness_countries.xlsx')
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for continent in continents:
        cou_atr = attr_within_continent(move_data=move_data, 
                                        continent=continent)
        cou_atr.to_excel(writer, sheet_name=continent)
    writer.close()

In [13]:
writer = pd.ExcelWriter('data/Double_verified_results/attractiveness_cities.xlsx')
all_countries = move_data['country_code'].unique()

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for country in all_countries:
        city_atr = attr_within_country(move_data, country)
        city_atr.to_excel(writer, sheet_name=country)
    writer.close()

In [14]:
ld_col = ['P(d=0)', 'P(d=1)', 'P(d=2)', 'P(d=3)']

writer = pd.ExcelWriter('data/Double_verified_results/level_distances.xlsx')

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for country in all_countries:
        ld_df = pd.DataFrame(columns=ld_col)
        country_movement = move_data[move_data['prev_country']==country]
        for year in range(1960, 2021, 5):
            slice_data = country_movement[ (country_movement['move_year'] >= year) 
                                & (country_movement['move_year']<(year+5))]
            sum = len(slice_data)
            pld = [len(slice_data[slice_data['level_distance'] == i])/sum if sum !=0 else 0
                for i in range(4)]
            pld = pd.DataFrame([pld], columns=ld_col, 
                            index=['%d-%d'%(year, year+5)])
            ld_df = pd.concat([ld_df, pld], axis=0)
        ld_df.to_excel(writer, sheet_name = country)
    writer.close()        